In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys
sys.path.append("../..")
sys.path.append("../../..")

# 1. Open dataset

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
path = "/data/pcpe/pcpe_03.csv"
df = pd.read_csv(path, delimiter=';', decimal=',')
df

/tmp/ipykernel_1798/1803422712.py:2: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=';', decimal=',')


,NUMERO_CASO,NUMERO_BANCO,NOME_BANCO,NUMERO_AGENCIA,NUMERO_CONTA,TIPO,CPF_CNPJ_TITULAR,NOME_TITULAR,DATA_LANCAMENTO,CPF_CNPJ_OD,...,VALOR_SALDO,NATUREZA_SALDO,NUMERO_BANCO_OD,NUMERO_AGENCIA_OD,NUMERO_CONTA_OD,NOME_ENDOSSANTE_CHEQUE,DOC_ENDOSSANTE_CHEQUE,DIA_LANCAMENTO,MES_LANCAMENTO,ANO_LANCAMENTO
0,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,2764,10081653,2,37469951830,MARTA VILLADICANI,2019-02-22 00:00:00,0,...,128065.54,C,0,0,NaN,NaN,NaN,22,2,2019
1,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,2764,10081653,2,37469951830,MARTA VILLADICANI,2019-02-27 00:00:00,0,...,131594.18,C,0,0,NaN,NaN,NaN,27,2,2019
2,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,2764,10081653,2,37469951830,MARTA VILLADICANI,2019-02-27 00:00:00,0,...,131688.04,C,0,0,NaN,NaN,NaN,27,2,2019
3,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,2764,10081653,2,37469951830,MARTA VILLADICANI,2019-04-02 00:00:00,0,...,135574.62,C,0,0,NaN,NaN,NaN,2,4,2019
4,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,2764,10081653,2,37469951830,MARTA VILLADICANI,2019-04-30 00:00:00,0,...,135819.30,C,0,0,NaN,NaN,NaN,30,4,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167027,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,3929,5251117,1,6745923440,MORENA GALEATI,2022-10-31 00:00:00,43808741000106,...,7066.60,C,403,1,4738204.0,NaN,NaN,31,10,2022
167028,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,3929,5251117,1,6745923440,MORENA GALEATI,2022-10-31 00:00:00,43808741000106,...,5323.10,C,403,1,4738204.0,NaN,NaN,31,10,2022
167029,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,3929,5251117,1,6745923440,MORENA GALEATI,2022-10-31 00:00:00,10585973458,...,9066.60,C,336,1,7924406.0,NaN,NaN,31,10,2022
167030,EXTRATO-2.0-F0R0B8,237,BANCO BRADESCO S.A,3929,5251117,1,6745923440,MORENA GALEATI,2022-10-31 00:00:00,10585973458,...,13145.15,C,336,1,7924406.0,NaN,NaN,31,10,2022


In [8]:
from publications.lacci2024.analysis import LacciAnalysis


# 2. Descriptive Statistics

In [10]:
lacci_analysis = LacciAnalysis(df) 
df_descriptive = lacci_analysis.get_results_descriptive()
print(df_descriptive.to_latex())


\begin{tabular}{lr}
\toprule
Typology & I-d \\
\midrule
Accounts & 169 \\
Individuals/Companies & 23 \\
Transactions & 167032 \\
\bottomrule
\end{tabular}



# 3. Data Spliting

In [14]:
from mlex import PastFutureSplit
split = PastFutureSplit()
X, y = lacci_analysis.get_X_y()
X_train, X_test, y_train, y_test = split.train_test_split(X, y)

# 4. Model Training

In [15]:
from mlex import (
    SimplePipeline,
    SimpleRNNModel,
    SimpleLSTMModel,
    SimpleGruModel,
)

In [16]:
from mlex import (
    SequenceTransfomer
)

In [25]:
import tensorflow as tf
sequence_length = 50
sequence = SequenceTransfomer(sequence_length=sequence_length)
data_test = sequence.transform(X_test, y_test)

data_train_full = sequence.transform(X_train, y_train)

N = len(data_train_full)
train_size = int(0.7 * N)
val_size = int(0.3 * N)

data_train = data_train_full.take(train_size)
data_val = data_train_full.skip(train_size)


In [ ]:
model_rnn = SimpleRNNModel(X_train.shape)
model_lstm = SimpleLSTMModel(X_train.shape)
model_gru = SimpleGruModel(X_train.shape)
import keras
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

pipelines = []
models = [model_rnn, model_lstm, model_gru]
# models = [model]

for m in models:
    pipeline = SimplePipeline(final_model=m.get_model())
    history = m.fit(data_train, epochs=30, validation_data=data_val, callbacks = [callback])
    pipelines.append(m)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


KeyError: 0

In [20]:
def get_y_pred_actual(y_pred_score, y_test, sequence_length):
    y_pred = y_pred_score > np.quantile(y_pred_score, 0.95)
    y_true = y_test[:-sequence_length+1]
    return y_true, y_pred
    

In [ ]:
list_y_pred_score = []
list_ys_true_pred = []
for p in pipelines:
    y_pred_score = p.predict(data_test)
    y_true, y_pred = get_y_pred_actual(y_pred_score, y_test, sequence_length)
    list_ys_true_pred.append((y_true, y_pred, y_pred_score))

In [14]:
from itertools import cycle
names = ['RNN', 'LSTM', 'GRU']
name_cycler = cycle(names)

In [ ]:
from publications.lacci2024.plotter import Plotter
plotter = Plotter()
for y_true, y_pred, y_pred_score in list_ys_true_pred:
    plotter.plot_matrix(y_true, y_pred,name_cycler, filename=f"confusion_{sequence_length}")


In [ ]:
from sklearn import metrics
import cycler
title = "ROC"
fig, ax = plt.subplots()
ax.plot([0,1], [0,1], "k--",linewidth=4, label='random classifier')
for y_true, y_pred, y_pred_score in list_ys_true_pred:
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_score)
    auc = metrics.auc(fpr, tpr)
    ax.plot(fpr, tpr,  linewidth=4, label=f"{next(name_cycler)} (AUC = {round(auc,2) })")
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel("False Positive Rate (1 - Specificity)", fontsize=16)
ax.set_ylabel("True Positive Rate (Sensitivity)", fontsize=16)
ax.set_title(f"Receiver Operating Characteristic \n {title}", fontsize=18)
ax.legend(loc="lower right")
plt.savefig(f"roc_{sequence_length}.pdf")

plt.show()

In [17]:
def roc(y_true, y_pred_score):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred_score)
    auc = metrics.auc(fpr, tpr)
    return auc

In [18]:
import scipy.stats as st

In [19]:
def roc_samples(y_t, y_s):
    N = len(y_t)
    rocs = []
    for b in range(30):
        choices = np.random.choice(N, size=N)
        ys_t = y_t[choices]
        ys_p = y_s[choices]
        fpr, tpr, thresholds = metrics.roc_curve(ys_t, ys_p)
        auc = metrics.auc(fpr, tpr)
        rocs.append(auc)
    return rocs

cis = []
for y_true, y_pred, y_pred_score in list_ys_true_pred:
    rocs = roc_samples(y_true, y_pred_score)
    roc_mean = np.mean(rocs)
    roc_std = np.std(rocs)
    roc_lb = roc_mean - 1.96 * roc_std
    roc_up = roc_mean + 1.96 * roc_std
    ci = f'{round(roc_mean, 2)} {round(roc_lb, 2), round(roc_up, 2)}'
    cis.append(ci)


In [ ]:
cis

In [25]:
import pandas as pd

In [ ]:
df_ci = pd.DataFrame({
        "Model": names,
        "95% Confidence Interval": cis
    })
print(df_ci.to_latex(index=False, escape=True))